# Tube Learning Notebook

## DataFrame Construction

In [4]:
import pandas as pd
import ast
import numpy as np
import glob

In [5]:
# Function to safely evaluate lists
def safe_eval(col):
    try:
        return ast.literal_eval(col)
    except ValueError:
        return col  # Return as is if it's not a string representation of a list

# Initialize an empty DataFrame
all_data = pd.DataFrame()

# Set the number of robots you want to include
num_robots = 5  # Set the number of robots to include
robot_indices = list(range(num_robots))  # Generates a list [0, 1, 2, ..., num_robots-1]

# Use glob to find all the files that match the pattern
file_list = glob.glob('data/trajectory_data_*.csv')

# Loop through the files sorted to maintain the order
for filename in sorted(file_list):
    temp_df = pd.read_csv(filename)
    # Filter the DataFrame to only include rows where the robot_index is in the list of desired indices
    temp_df = temp_df[temp_df['robot_index'].isin(robot_indices)]
    # Apply transformations right after reading
    temp_df['joint_positions'] = temp_df['joint_positions'].apply(safe_eval)
    temp_df['joint_velocities'] = temp_df['joint_velocities'].apply(safe_eval)
    all_data = pd.concat([all_data, temp_df], ignore_index=True)


In [6]:
# Now create the derived columns
all_data['x_t'] = all_data.apply(lambda row: row['joint_positions'] + row['joint_velocities'], axis=1)
all_data['u_t'] = all_data.apply(lambda row: [row['velocity_x'], row['velocity_y']], axis=1)
all_data['z_t'] = all_data.apply(lambda row: [row['traj_x'], row['traj_y']], axis=1)
all_data['v_t'] = all_data.apply(lambda row: [row['reduced_command_x'], row['reduced_command_y']], axis=1)


# Since w_t and w_{t+1} are derived from calculations, no need for safe_eval
all_data['w_t'] = np.sqrt((all_data['position_x'] - all_data['traj_x'])**2 + (all_data['position_y'] - all_data['traj_y'])**2)
all_data['group'] = all_data['episode_number'].astype(str) + '_' + all_data['robot_index'].astype(str)

# Example to debug with a smaller subset
all_data['x_{t+1}'] = all_data.groupby('group')['x_t'].shift(-1)
all_data['z_{t+1}'] = all_data.groupby('group')['z_t'].shift(-1)
all_data['w_{t+1}'] = all_data.groupby('group')['w_t'].shift(-1)

# Function to drop the first and last 10 data points from each episode
def drop_edges(group):
    return group.iloc[10:-10]
all_data = all_data.groupby('group', group_keys=False).apply(drop_edges)

# Drop rows where x_{t+1}, z_{t+1}, and w_{t+1} do not exist
all_data.dropna(subset=['x_{t+1}', 'z_{t+1}', 'w_{t+1}'], inplace=True)

# Select and order the final columns
final_df = all_data[['group', 'x_t', 'u_t', 'z_t', 'v_t', 'w_t', 'x_{t+1}', 'z_{t+1}', 'w_{t+1}']]

ValueError: Expected a 1D array, got an array with shape (0, 2)

We have $D=\{\omega_t, x_t, u_t, z_t, v_t, \omega_{t+1}, x_{t+1}, z_{t+1}\}$:

In [124]:
# Print the final DataFrame
final_df.head()

,group,x_t,u_t,z_t,v_t,w_t,x_{t+1},z_{t+1},w_{t+1}
50,1.0_0,"[0.021700723096728325, 0.5511646866798401, -0....","[0.5442334846271863, -0.283746258114005]","[0.0839964397055372, -0.0443801499923683]","[0.4199822079150199, -0.2219007549217102]",0.026090,"[0.023384274914860725, 0.5582820773124695, -0....","[0.0923960836760909, -0.0488181649916052]",0.029730
55,1.0_0,"[0.023384274914860725, 0.5582820773124695, -0....","[0.5480042460848393, -0.2824143952132594]","[0.0923960836760909, -0.0488181649916052]","[0.4199822079150199, -0.2219007549217102]",0.029730,"[0.02588886395096779, 0.5638883709907532, -0.7...","[0.1007957276466446, -0.053256179990842]",0.033340
60,1.0_0,"[0.02588886395096779, 0.5638883709907532, -0.7...","[0.550880471042393, -0.2824574469904211]","[0.1007957276466446, -0.053256179990842]","[0.4199822079150199, -0.2219007549217102]",0.033340,"[0.028353633359074593, 0.5677175521850586, -0....","[0.1091953716171983, -0.0576941949900788]",0.036844
65,1.0_0,"[0.028353633359074593, 0.5677175521850586, -0....","[0.5525190403579519, -0.282065262418896]","[0.1091953716171983, -0.0576941949900788]","[0.4199822079150199, -0.2219007549217102]",0.036844,"[0.030368400737643242, 0.5698843002319336, -0....","[0.1175950155877521, -0.0621322099893157]",0.040191
70,1.0_0,"[0.030368400737643242, 0.5698843002319336, -0....","[0.5536589812999309, -0.2793862519163999]","[0.1175950155877521, -0.0621322099893157]","[0.4199822079150199, -0.2219007549217102]",0.040191,"[0.031975340098142624, 0.5703613758087158, -0....","[0.1259946595583058, -0.0665702249885525]",0.043377


Optional Saving:

In [125]:
final_df.to_csv('processed_trajectory_data.csv', index=False)

## Network Construction

In [18]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import ast
from tqdm import tqdm
import wandb

wandb.login(key="70954bb73c536b7f5b23ef315c7c19b511e8a406")

wandb: Currently logged in as: coleonguard (coleonguard-Georgia Institute of Technology). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/colejohnson/.netrc


True

Load in the data:

In [19]:
def safe_eval(col):
    try:
        return ast.literal_eval(col)
    except ValueError:
        return col  # Return as is if it's not a string representation of a list

# Load data (assuming DataFrame is saved in a CSV file named 'processed_trajectory_data.csv')
final_df = pd.read_csv('processed_trajectory_data.csv')

# Apply safe_eval to columns that are expected to contain lists or lists of lists
list_columns = ['x_t', 'u_t', 'z_t', 'v_t']
for col in list_columns:
    final_df[col] = final_df[col].apply(safe_eval)

# Convert to tensors
# You might need to first convert list columns into the proper format for tensor conversion
def convert_to_tensor_input(row):
    flat_list = [item for sublist in row for item in sublist]  # Flattens a list of lists if necessary
    return flat_list

X = torch.tensor(final_df[list_columns].apply(convert_to_tensor_input, axis=1).tolist(), dtype=torch.float32)
y = torch.tensor(final_df[['w_t', 'w_{t+1}']].values, dtype=torch.float32)

# Create TensorDataset
dataset = TensorDataset(X, y[:, 1].unsqueeze(1))

# Split data into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoader objects
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

Model specifications:

In [20]:
class TubeWidthPredictor(nn.Module):
    def __init__(self, input_size=30, num_units=64, num_layers=2):
        super(TubeWidthPredictor, self).__init__()
        layers = [nn.Linear(input_size, num_units), nn.ReLU()]
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_units, num_units), nn.ReLU()]
        layers.append(nn.Linear(num_units, 1))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

class AsymmetricLoss(nn.Module):
    def __init__(self, alpha=0.9, delta=1.0):
        super(AsymmetricLoss, self).__init__()
        self.alpha = alpha
        self.huber = nn.HuberLoss(delta=delta)
    
    def forward(self, y_pred, y_true):
        residual = y_true - y_pred
        loss = torch.where(residual > 0, self.alpha * residual, (1 - self.alpha) * residual.abs())
        return self.huber(loss, torch.zeros_like(loss))

def train_and_test(model, criterion, optimizer, train_loader, test_loader, num_epochs=500):
    for epoch in tqdm(range(num_epochs), desc="Epochs"):
        model.train()
        train_loss = 0
        for data, targets in tqdm(train_loader, desc="Training Batches", leave=False):
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)

        model.eval()
        total_loss = 0
        with torch.no_grad():
            for data, targets in tqdm(test_loader, desc="Testing Batches", leave=False):
                outputs = model(data)
                loss = criterion(outputs, targets)
                total_loss += loss.item()
        test_loss = total_loss / len(test_loader)

        wandb.log({'Train Loss': train_loss, 'Test Loss': test_loss, 'Epoch': epoch})

In [21]:
def main():
    wandb.init()
    config = wandb.config

    model = TubeWidthPredictor(input_size=30, num_units=config.num_units, num_layers=config.num_layers)
    criterion = AsymmetricLoss(alpha=config.alpha, delta=1.0)
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    train_loader = DataLoader(...)  # Define your dataset and dataloader
    test_loader = DataLoader(...)

    train_and_test(model, criterion, optimizer, train_loader, test_loader, num_epochs=500)
    wandb.finish()

In [ ]:
alpha_values = [0.8, 0.9, 0.95, 0.99, 0.999]
for alpha in alpha_values:
    sweep_config = {
        'method': 'grid',
        'metric': {
            'name': 'Test Loss',
            'goal': 'minimize'
        },
        'parameters': {
            'alpha': {
                'value': alpha
            },
            'learning_rate': {
                'values': [0.001, 0.01, 0.1]
            },
            'num_units': {
                'values': [32, 64, 128]
            },
            'num_layers': {
                'values': [1, 2, 3]
            }
        }
    }
    sweep_id = wandb.sweep(sweep_config, project=f"tube_width_experiment_alpha_{alpha}")
    wandb.agent(sweep_id, main)


Create sweep with ID: ub8qh2pq
Sweep URL: https://wandb.ai/coleonguard-Georgia%20Institute%20of%20Technology/tube_width_experiment_alpha_0.8/sweeps/ub8qh2pq


wandb: Agent Starting Run: masxxj15 with config:
wandb: 	alpha: 0.8
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 1
wandb: 	num_units: 32


Epochs:   0%|                                           | 0/500 [00:00<?, ?it/s]
Training Batches: 0it [00:00, ?it/s]
Epochs:   0%|                                           | 0/500 [00:00<?, ?it/s]


Run masxxj15 errored:
Traceback (most recent call last):
  File "/Users/colejohnson/opt/anaconda3/lib/python3.9/site-packages/wandb/agents/pyagent.py", line 307, in _run_job
    self._function()
  File "/var/folders/xn/h81hn6wd44q8p6gmpfy06f6w0000gn/T/ipykernel_51281/1440170456.py", line 12, in main
    train_and_test(model, criterion, optimizer, train_loader, test_loader, num_epochs=500)
  File "/var/folders/xn/h81hn6wd44q8p6gmpfy06f6w0000gn/T/ipykernel_51281/1750271069.py", line 28, in train_and_test
    for data, targets in tqdm(train_loader, desc="Training Batches", leave=False):
  File "/Users/colejohnson/opt/anaconda3/lib/python3.9/site-packages/tqdm/std.py", line 1195, in __iter__
    for obj in iterable:
  File "/Users/colejohnson/opt/anaconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 633, in __next__
    data = self._next_data()
  File "/Users/colejohnson/opt/anaconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 676, in _next_data

Epochs:   0%|                                           | 0/500 [00:00<?, ?it/s]
Training Batches: 0it [00:00, ?it/s]
Epochs:   0%|                                           | 0/500 [00:00<?, ?it/s]


Run ozyguigi errored:
Traceback (most recent call last):
  File "/Users/colejohnson/opt/anaconda3/lib/python3.9/site-packages/wandb/agents/pyagent.py", line 307, in _run_job
    self._function()
  File "/var/folders/xn/h81hn6wd44q8p6gmpfy06f6w0000gn/T/ipykernel_51281/1440170456.py", line 12, in main
    train_and_test(model, criterion, optimizer, train_loader, test_loader, num_epochs=500)
  File "/var/folders/xn/h81hn6wd44q8p6gmpfy06f6w0000gn/T/ipykernel_51281/1750271069.py", line 28, in train_and_test
    for data, targets in tqdm(train_loader, desc="Training Batches", leave=False):
  File "/Users/colejohnson/opt/anaconda3/lib/python3.9/site-packages/tqdm/std.py", line 1195, in __iter__
    for obj in iterable:
  File "/Users/colejohnson/opt/anaconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 633, in __next__
    data = self._next_data()
  File "/Users/colejohnson/opt/anaconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 676, in _next_data

Epochs:   0%|                                           | 0/500 [00:00<?, ?it/s]
Training Batches: 0it [00:00, ?it/s]
Epochs:   0%|                                           | 0/500 [00:00<?, ?it/s]


Run 2idpcuvi errored:
Traceback (most recent call last):
  File "/Users/colejohnson/opt/anaconda3/lib/python3.9/site-packages/wandb/agents/pyagent.py", line 307, in _run_job
    self._function()
  File "/var/folders/xn/h81hn6wd44q8p6gmpfy06f6w0000gn/T/ipykernel_51281/1440170456.py", line 12, in main
    train_and_test(model, criterion, optimizer, train_loader, test_loader, num_epochs=500)
  File "/var/folders/xn/h81hn6wd44q8p6gmpfy06f6w0000gn/T/ipykernel_51281/1750271069.py", line 28, in train_and_test
    for data, targets in tqdm(train_loader, desc="Training Batches", leave=False):
  File "/Users/colejohnson/opt/anaconda3/lib/python3.9/site-packages/tqdm/std.py", line 1195, in __iter__
    for obj in iterable:
  File "/Users/colejohnson/opt/anaconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 633, in __next__
    data = self._next_data()
  File "/Users/colejohnson/opt/anaconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 676, in _next_data